In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [3]:
class batsmanState(TypedDict):
    runs : int
    balls :int
    fours:int
    six:int

    sr : float
    bpb : float
    boundary_percent : float
    summary : str


In [4]:
def calculate_strike_rate(state:batsmanState):
    sr = (state['runs'] / state['balls'])/100

    return {'sr' : sr}

In [5]:
def calculate_boundary_per_ball(state:batsmanState):
    bp = state['balls'] / (state['fours'] + state['six'])

    return {'bpb' : bp}

In [6]:
def calculate_boundary_percent(state:batsmanState):
    per = ((state['fours'] * 4 + state['six'] * 6) / state['balls']) * 100 

    return {'boundary_percent': per}

In [7]:
def summary(state:batsmanState):

    summary = f"""
            Strike Rate : {state['sr']} \n
            Boundary Per Ball : {state['bpb']} \n
            Boundary Percent : {state['boundary_percent']}
"""
    return {'summary':summary}

In [8]:
graph = StateGraph(batsmanState)
graph.add_node('calculate_strike_rate',calculate_strike_rate)
graph.add_node('calculate_boundary_per_ball',calculate_boundary_per_ball)
graph.add_node('calculate_boundary_percent',calculate_boundary_percent)
graph.add_node('summary',summary)

In [9]:
# edges

graph.add_edge(START,'calculate_strike_rate')
graph.add_edge(START,'calculate_boundary_per_ball')
graph.add_edge(START,'calculate_boundary_percent')
graph.add_edge('calculate_strike_rate','summary')
graph.add_edge('calculate_boundary_per_ball','summary')
graph.add_edge('calculate_boundary_percent','summary')
graph.add_edge('summary',END)

In [10]:
workflow = graph.compile()

In [11]:
initial_state = {
    'runs': 100,
    'balls': 50,
    'fours': 6,
    'six': 4
}

result = workflow.invoke(initial_state)
print(result['summary'])
# print(result)



            Strike Rate : 0.02 

            Boundary Per Ball : 5.0 

            Boundary Percent : 96.0

